In [ ]:
import pandas as pd
import numpy as np
import datetime
# Read the CSV file into a DataFrame
file_path = r'C:\Users\LeAnhNguyen\Desktop\Obeta\003 pick_data.csv'
dtype_spec = {'product_id': int, 'warehouse_section': str, 'origin': int,␣
↪'order_number': int, 'position_in_order': str, 'pick_volume': int,␣
↪'quantity_unit': str, 'date': str}
df = pd.read_csv(file_path, header = None , dtype=dtype_spec, low_memory=False)
# Name the header
df.columns = ["product_id", "warehouse_section", "origin", "order_number",␣
↪"position_in_order","pick_volume","quantity_unit","date"]
df.head()

In [ ]:
#Check number of rows and columns
df.shape

In [ ]:
#remove duplicate
df = df.drop_duplicates()
#Check number of rows and columns after remove duplicate
print(df.shape)

In [ ]:
# Remove rows where pick volume = negative or pick volume = 0
df = df[df['pick_volume']> 0]
print(df)
##Check number of rows and columns after remove pick volume <= 0
print(df.shape)

In [ ]:
# Define a function to identify outliers by using 3 Standard Deviations method
def find_group_outliers(group):
mean = group.mean()
std = group.std()
threshold = 3 * std
outliers = group[abs(group - mean) > threshold]
return len(outliers), outliers.index.tolist() # Return the count and␣
↪indices of outliers
# Use groupby to apply the function to each product_id
grouped_outliers_info = df.groupby('product_id')['pick_volume'].apply(find_group_outliers)
# Create a list of row number(index of row) of outliers, It will be refered to when erasing outliers
all_outliers_indices = [index for _, indices in grouped_outliers_info for index in indices]
#count the total number of outliers
total_outliers_count = sum(count for count, _ in grouped_outliers_info)
# Erase the rows containing outliers
df_cleaned = df.drop(index=all_outliers_indices)
# Show the cleaned DataFrame and total number of outliers
print("Cleaned DataFrame:")
print(df_cleaned)
print("\nTotal number of outliers:", total_outliers_count)
#Check number of rows and columns after remove outliers
print('\nShape of the cleaned df:',df_cleaned.shape)

In [ ]:
# Function for univariate analysis of nominal columns
def univariate_nominal(group):
count = group.count()
unique_count = group.nunique()
mode_value = group.mode().iloc[0]
mode_frequency = group.value_counts().iloc[0] # Add this line to calculate␣
↪mode frequency
return count, unique_count, mode_value, mode_frequency
# List of columns to analyze
columns_to_analyze = ['product_id', 'warehouse_section', 'origin','order_number', 'quantity_unit']
# Dictionary to store results
results_nominal = {}
# Applying the function to each column
for column in columns_to_analyze:
results_nominal[column] = univariate_nominal(df_cleaned[column])
# Displaying the results
for column, result in results_nominal.items():
print(f"Univariate Analysis for {column}:")
print(f"Count: {result[0]}")
print(f"Unique Count: {result[1]}")
print(f"Mode Value: {result[2]}")
print(f"Mode Frequency: {result[3]}")
print("\n")

In [ ]:
# Function for univariate analysis of pick_vol column
def descriptive_stats_numeric(group):
stats = {
'Count': group.count(),
'Unique Count': group.nunique(),
'Maximum': group.max(),
'Minimum': group.min(),
'Median': group.median(),
'Mode': group.mode().iloc[0] if not group.mode().empty else None,
'Mode Frequency': group.value_counts().max() if not group.mode().empty
    else None,
'Mean': group.mean(), # Added line for mean calculation
'25%': group.quantile(0.25),
'50%': group.quantile(0.5),
'75%': group.quantile(0.75),
'IQR': group.quantile(0.75) - group.quantile(0.25),
'Std': group.std(),
'Variance': group.var(),
}
return stats
# Apply the function to the pick_volume column
numeric_stats_result = descriptive_stats_numeric(df_cleaned['pick_volume'])
# Show the result
print("Descriptive Statistics for Numeric Variable:")
print(numeric_stats_result)

In [ ]:
: #Create a new order number column with prefix because of repeating order number issue
# Convert 'order_date' to datetime format
df_cleaned['date'] = pd.to_datetime(df_cleaned['date'])
# Extract the year from 'order_date'
df_cleaned['year'] = df_cleaned['date'].dt.year
# Create the 'adjusted order number' column with the prefix of the year
df_cleaned['adjusted_order_number'] = df_cleaned['year'].astype(str) + '_' + df_cleaned['order_number'].astype(str)
# Displaying the DataFrame with the new column
df_cleaned

In [ ]:
# Read the 2nd CSV file into a DataFrame
file_path = r'C:\Users\LeAnhNguyen\Desktop\Obeta\002 product_data UTF8.txt'
df1 = pd.read_csv(file_path)
df1.columns = ["product_id", "description", "product_group"]
df1

In [ ]:
# Replace German umlauts in the 'City' column
df1['product_group'] = df1['product_group'].str.replace('ü', 'ue').str.replace('ö', 'oe').str.replace('ä', 'ae')
df1['description'] = df1['description'].str.replace('ü', 'ue').str.replace('ö','oe').str.replace('ä', 'ae')
# Display the result
print(df1)

In [ ]:
# Build ER model
# There are 1 Fact Table named 'Aggregated Fact Table', and 2 Dimensional Tables named, Product Table, and Order Table.
# Steps to create the Product Table
# Filter by columns 'product_id', 'quantity_unit'
columns_to_keep = ['product_id', 'quantity_unit']
# Select specific columns from df_cleaned
df_cleaned_selected = df_cleaned[columns_to_keep].copy()
# Remove duplicates based on the selected columns
df_cleaned_selected = df_cleaned_selected.drop_duplicates()
# Merge with df1 using left join, df1 is the 2nd imported .csv file which is␣
↪product data
product_1st_draft = df1.merge(df_cleaned_selected, how='left', on='product_id')
# Show 1st draft of Product Table
product_1st_draft

In [ ]:
 # Steps to create the Order Table
# Filter by columns 'adjusted_order_number','order_number','origin','position_in_order'
columns_to_keep1 = ['adjusted_order_number','order_number','origin','position_in_order']
# Select specific columns from df_cleaned
order_no_dup = df_cleaned[columns_to_keep1].copy()
# Remove duplicates based on the selected columns
order_no_dup = order_no_dup.drop_duplicates()
# Show 1st draft of Order Table
order_no_dup

In [ ]:
order= order_no_dup
#Concatenate 3 columns from df1 and 1 column from product_1st_draft
product= pd.concat([df1['product_id'], product_1st_draft['quantity_unit'],df1['product_group'],df1['description']], axis=1)
# Displaying the separated tables
print("\norder:")
print(order)
print("\nproduct:")
print(product)

In [ ]:
: # Create the Aggregated Fact Table
# Create 2 new columns to help minimizing Tableau computing time
aggregated_pick = df_cleaned.groupby(['adjusted_order_number', 'product_id','date', 'warehouse_section']).agg(
sum_pick_volume=pd.NamedAgg(column='pick_volume', aggfunc='sum'),
count_order_number=pd.NamedAgg(column='adjusted_order_number',aggfunc='count')
).reset_index()
aggregated_pick

In [ ]:
# Convert 'date' column to datetime type
aggregated_pick['date'] = pd.to_datetime(aggregated_pick['date'])
# Sort the DataFrame by 'order_number' and 'date'
aggregated_pick = aggregated_pick.sort_values(by=['adjusted_order_number','date','warehouse_section'])
# Calculate the lead time between consecutive picks for each 'order_number' and 'product_id'
aggregated_pick['lead_time'] = aggregated_pick.groupby(['adjusted_order_number', 'warehouse_section'])['date'].diff().dt.total_seconds()
# Calculate the average lead time per unit pick volume for each 'product_id'
aggregated_pick['avg_lead_time_per_pick'] = aggregated_pick['lead_time'] / aggregated_pick['sum_pick_volume']
# Display the DataFrame with lead times
aggregated_pick


In [ ]:
# Merge 'product' with 'avg_lead_time_per_pick_by_product' based on 'product_id'
product = product.merge(
aggregated_pick.groupby('product_id')['avg_lead_time_per_pick'].mean().reset_index(),
how='left',
on='product_id',
suffixes=('', '_new') # Use a suffix to distinguish the columns
)
# Drop unnecessary column
product = product.drop(columns=['avg_lead_time_per_pick'])
# Display the result
product

In [ ]:
# Filter by these columns
pick_columns = ['adjusted_order_number','product_id','date','warehouse_section',
'sum_pick_volume','count_order_number','lead_time',]
agg_pick= aggregated_pick[pick_columns]
# Display Aggregated Fact Table
agg_pick

In [ ]:
# Prepare for connecting to MySQL
!pip install mysql-connector-python
import mysql.connector
from mysql.connector import Error
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine

In [ ]:
host_name = 'localhost'
db_name = 'obeta_production'
u_name = 'root'
u_pass = 'mysql'
port_num = 3306
# Convert port_num to a string
port_num_str = str(port_num)
#create_engine('mysql+mysqlconnector://' + [root]:[mysql]@[localhost]:[3306]/[obeta_production]',echo=False)
my_eng = create_engine('mysql+mysqlconnector://' + u_name + ':'+ u_pass + '@' + host_name + ':' + port_num_str + '/' + db_name ,echo=False)
Session = sessionmaker(bind=my_eng)
session = Session()
try:
# Perform database operations within the transaction
product.to_sql(name='product', con=my_eng, if_exists = 'append', index = False, chunksize=1000)
agg_pick.to_sql(name='agg_pick', con=my_eng, if_exists = 'append', index = False, chunksize=1000)
order.to_sql(name='orders', con=my_eng, if_exists = 'append', index = False, chunksize=1000)
# always commit changes!
session.commit()
except: # Handle any exceptions that occur during the transaction
# Explicitly rollback the transaction
if session:
session.rollback()
raise
finally:
# Close the connection
if session:
session.close()
